In [1]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import matplotlib.font_manager as fm
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

font_location = 'C:/Windows/Fonts/malgunsl.ttf'
font_name = fm.FontProperties(fname = font_location).get_name()
sns.set(rc = {'figure.figsize':(15,8)})
matplotlib.rc('font' , family=font_name)#한글폰트 설정

In [95]:
# 데이터 로드
df_car =pd.read_excel('data_2/자동차항만입출항데이터.xlsx', index_col=0)
load_in =pd.read_excel('data_2/외항입항_화물료데이터.xlsx', index_col=0)
load_out =pd.read_excel('data_2/외항출항_화물료데이터.xlsx', index_col=0)
df_ton =pd.read_excel('data_2/입출항적재량.xlsx', index_col=0)

# 자동차 입출항 데이터 전처리

In [97]:
df_car

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,국제톤수,...,차항지,전출항지,선박용도,승무원 1,승무원 2,승객,예선,도선,부선호출부호1,부선호출부호2
0,울산,9HA2293,TULANE,2012,1,최종,외항,출항,"72,295","72,295",...,NaN,NaN,자동차운반선,NaN,24.0,1.0,Y,Y,NaN,NaN
1,울산,C6ZE7,GLOVIS CARAVEL,2012,1,최종,외항,출항,"58,767","58,767",...,NaN,NaN,자동차운반선,NaN,24.0,4.0,Y,Y,NaN,NaN
2,울산,C6ZE8,GLOVIS CLIPPER,2012,1,최종,외항,출항,"58,767","58,767",...,NaN,NaN,자동차운반선,NaN,24.0,NaN,Y,Y,NaN,NaN
3,울산,C6ZS3,GLOVIS MERMAID,2012,1,최종,외항,출항,"32,545","32,545",...,NaN,NaN,자동차운반선,NaN,21.0,NaN,Y,Y,NaN,NaN
4,울산,C6ZS5,GLOVIS MELODY,2012,1,최종,외항,출항,"32,545","32,545",...,NaN,NaN,자동차운반선,NaN,20.0,NaN,Y,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11457,울산,H3NG,PRESTIGE ACE,2021,2,최초,외항,입항,"55,878","55,878",...,광양,SINGAPORE,자동차운반선,NaN,21.0,NaN,Y,Y,NaN,NaN
11458,울산,ZDNC6,PASSAMA,2021,2,최초,외항,입항,"47,053","47,053",...,MOJI/KITAKYUSHU,마산,자동차운반선,NaN,20.0,NaN,Y,Y,NaN,NaN
11459,울산,DSMZ8,아세안 메저스티호,2021,2,변경,외항,입항,"71,383","71,383",...,평택.당진,마산,자동차운반선,9.0,14.0,NaN,Y,Y,NaN,NaN
11460,울산,3FDI5,MORNING LILY,2021,2,변경,외항,입항,"70,853","70,853",...,마산,평택.당진,자동차운반선,10.0,14.0,NaN,Y,Y,NaN,NaN


In [98]:
# 필요없는 변수 제거
df_car.drop(['항명', '구분', '외내', '국제톤수', '징수톤수', 'CIO수속일자', '수리일시', '선박용도', '항해구분', '국적', '국적.1', '승무원 1', '승무원 2', '승객', '부선호출부호1', '부선호출부호2', '계선장소.2'], axis=1, inplace=True)
df_car

,호출부호,선명,입항횟수,입항횟수.1,입출,총톤수,입항일시,출항일시,MRN 번호,계선장소,계선장소.1,차항지,전출항지,예선,도선
0,9HA2293,TULANE,2012,1,출항,"72,295",1950-01-01 00:00,2012-06-15 15:00,12WLHNU0283,MQP,3,NaN,NaN,Y,Y
1,C6ZE7,GLOVIS CARAVEL,2012,1,출항,"58,767",1950-01-01 00:00,2012-09-10 15:05,12GLVSU1531,MBJ,3,NaN,NaN,Y,Y
2,C6ZE8,GLOVIS CLIPPER,2012,1,출항,"58,767",1950-01-01 00:00,2012-02-18 13:00,12GLVSU0295,MBJ,2,NaN,NaN,Y,Y
3,C6ZS3,GLOVIS MERMAID,2012,1,출항,"32,545",1950-01-01 00:00,2012-07-05 17:05,12STKOD0941,MQM,3,NaN,NaN,Y,Y
4,C6ZS5,GLOVIS MELODY,2012,1,출항,"32,545",1950-01-01 00:00,2012-11-22 09:35,12STKOD1683,MQM,3,NaN,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11457,H3NG,PRESTIGE ACE,2021,2,입항,"55,878",2021-08-16 06:15,2021-08-16 22:00,21GLVSE147I,MBJ,1,광양,SINGAPORE,Y,Y
11458,ZDNC6,PASSAMA,2021,2,입항,"47,053",2021-08-16 08:00,2021-08-19 09:00,21L53KI448I,MBJ,3,MOJI/KITAKYUSHU,마산,Y,Y
11459,DSMZ8,아세안 메저스티호,2021,2,입항,"71,383",2021-08-17 06:15,2021-08-18 22:00,21EUKOV913I,MBJ,2,평택.당진,마산,Y,Y
11460,3FDI5,MORNING LILY,2021,2,입항,"70,853",2021-08-18 06:15,2021-08-18 22:00,21EUKOV921I,MBJ,3,마산,평택.당진,Y,Y


In [99]:
# 입출 칼럼 입항:1, 출항:0 으로 변환

# dict: 기존 값, 바꾸고자 하는 값
change_dict = {'입항': 1, '출항': 0}

# 변경
df_car = df_car.replace({'입출':change_dict})
df_car

,호출부호,선명,입항횟수,입항횟수.1,입출,총톤수,입항일시,출항일시,MRN 번호,계선장소,계선장소.1,차항지,전출항지,예선,도선
0,9HA2293,TULANE,2012,1,0,"72,295",1950-01-01 00:00,2012-06-15 15:00,12WLHNU0283,MQP,3,NaN,NaN,Y,Y
1,C6ZE7,GLOVIS CARAVEL,2012,1,0,"58,767",1950-01-01 00:00,2012-09-10 15:05,12GLVSU1531,MBJ,3,NaN,NaN,Y,Y
2,C6ZE8,GLOVIS CLIPPER,2012,1,0,"58,767",1950-01-01 00:00,2012-02-18 13:00,12GLVSU0295,MBJ,2,NaN,NaN,Y,Y
3,C6ZS3,GLOVIS MERMAID,2012,1,0,"32,545",1950-01-01 00:00,2012-07-05 17:05,12STKOD0941,MQM,3,NaN,NaN,Y,Y
4,C6ZS5,GLOVIS MELODY,2012,1,0,"32,545",1950-01-01 00:00,2012-11-22 09:35,12STKOD1683,MQM,3,NaN,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11457,H3NG,PRESTIGE ACE,2021,2,1,"55,878",2021-08-16 06:15,2021-08-16 22:00,21GLVSE147I,MBJ,1,광양,SINGAPORE,Y,Y
11458,ZDNC6,PASSAMA,2021,2,1,"47,053",2021-08-16 08:00,2021-08-19 09:00,21L53KI448I,MBJ,3,MOJI/KITAKYUSHU,마산,Y,Y
11459,DSMZ8,아세안 메저스티호,2021,2,1,"71,383",2021-08-17 06:15,2021-08-18 22:00,21EUKOV913I,MBJ,2,평택.당진,마산,Y,Y
11460,3FDI5,MORNING LILY,2021,2,1,"70,853",2021-08-18 06:15,2021-08-18 22:00,21EUKOV921I,MBJ,3,마산,평택.당진,Y,Y


In [100]:
# 칼럼 이름 변경
df_car.rename(columns={'입항횟수.1': '입항횟수', '입항횟수':'입항년도'}, inplace=True)
df_car

,호출부호,선명,입항년도,입항횟수,입출,총톤수,입항일시,출항일시,MRN 번호,계선장소,계선장소.1,차항지,전출항지,예선,도선
0,9HA2293,TULANE,2012,1,0,"72,295",1950-01-01 00:00,2012-06-15 15:00,12WLHNU0283,MQP,3,NaN,NaN,Y,Y
1,C6ZE7,GLOVIS CARAVEL,2012,1,0,"58,767",1950-01-01 00:00,2012-09-10 15:05,12GLVSU1531,MBJ,3,NaN,NaN,Y,Y
2,C6ZE8,GLOVIS CLIPPER,2012,1,0,"58,767",1950-01-01 00:00,2012-02-18 13:00,12GLVSU0295,MBJ,2,NaN,NaN,Y,Y
3,C6ZS3,GLOVIS MERMAID,2012,1,0,"32,545",1950-01-01 00:00,2012-07-05 17:05,12STKOD0941,MQM,3,NaN,NaN,Y,Y
4,C6ZS5,GLOVIS MELODY,2012,1,0,"32,545",1950-01-01 00:00,2012-11-22 09:35,12STKOD1683,MQM,3,NaN,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11457,H3NG,PRESTIGE ACE,2021,2,1,"55,878",2021-08-16 06:15,2021-08-16 22:00,21GLVSE147I,MBJ,1,광양,SINGAPORE,Y,Y
11458,ZDNC6,PASSAMA,2021,2,1,"47,053",2021-08-16 08:00,2021-08-19 09:00,21L53KI448I,MBJ,3,MOJI/KITAKYUSHU,마산,Y,Y
11459,DSMZ8,아세안 메저스티호,2021,2,1,"71,383",2021-08-17 06:15,2021-08-18 22:00,21EUKOV913I,MBJ,2,평택.당진,마산,Y,Y
11460,3FDI5,MORNING LILY,2021,2,1,"70,853",2021-08-18 06:15,2021-08-18 22:00,21EUKOV921I,MBJ,3,마산,평택.당진,Y,Y


In [101]:
Out = pd.DataFrame(df_car.query('입출 == 0'))
In = pd.DataFrame(df_car.query('입출 == 1'))

In [102]:
Out

,호출부호,선명,입항년도,입항횟수,입출,총톤수,입항일시,출항일시,MRN 번호,계선장소,계선장소.1,차항지,전출항지,예선,도선
0,9HA2293,TULANE,2012,1,0,"72,295",1950-01-01 00:00,2012-06-15 15:00,12WLHNU0283,MQP,3,NaN,NaN,Y,Y
1,C6ZE7,GLOVIS CARAVEL,2012,1,0,"58,767",1950-01-01 00:00,2012-09-10 15:05,12GLVSU1531,MBJ,3,NaN,NaN,Y,Y
2,C6ZE8,GLOVIS CLIPPER,2012,1,0,"58,767",1950-01-01 00:00,2012-02-18 13:00,12GLVSU0295,MBJ,2,NaN,NaN,Y,Y
3,C6ZS3,GLOVIS MERMAID,2012,1,0,"32,545",1950-01-01 00:00,2012-07-05 17:05,12STKOD0941,MQM,3,NaN,NaN,Y,Y
4,C6ZS5,GLOVIS MELODY,2012,1,0,"32,545",1950-01-01 00:00,2012-11-22 09:35,12STKOD1683,MQM,3,NaN,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11442,D5DX5,CSAV RIO NEVADO,2021,4,0,"46,800",2021-07-31 12:25,2021-07-31 22:30,21GLVSU144E,MBJ,1,광양,목포,Y,Y
11444,9V9825,TURANDOT,2021,4,0,"55,598",2021-08-02 07:05,2021-08-02 17:00,21L53KE428E,MBP,1,OSAKA,평택.당진,Y,Y
11445,MMNA2,MORNING MIDAS,2021,1,0,"46,800",2021-08-11 06:15,2021-08-11 22:00,21EUKOV893E,MBJ,1,광양,SINGAPORE,Y,Y
11447,C6XA4,RCC TIANJIN,2021,3,0,"41,009",2021-08-11 07:48,2021-08-10 22:00,21GLVSU145E,MBJ,2,광양,NEW WESTMINSTER,Y,Y


In [103]:
Out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5740 entries, 0 to 11449
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   호출부호    5740 non-null   object
 1   선명      5740 non-null   object
 2   입항년도    5740 non-null   int64 
 3   입항횟수    5740 non-null   int64 
 4   입출      5740 non-null   int64 
 5   총톤수     5740 non-null   object
 6   입항일시    5740 non-null   object
 7   출항일시    5740 non-null   object
 8   MRN 번호  5740 non-null   object
 9   계선장소    5740 non-null   object
 10  계선장소.1  5740 non-null   int64 
 11  차항지     5715 non-null   object
 12  전출항지    5715 non-null   object
 13  예선      5740 non-null   object
 14  도선      5740 non-null   object
dtypes: int64(4), object(11)
memory usage: 717.5+ KB


In [105]:
Out.isnull().sum()

호출부호       0
선명         0
입항년도       0
입항횟수       0
입출         0
총톤수        0
입항일시       0
출항일시       0
MRN 번호     0
계선장소       0
계선장소.1     0
차항지       25
전출항지      25
예선         0
도선         0
dtype: int64

In [104]:
In.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5722 entries, 14 to 11461
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   호출부호    5722 non-null   object
 1   선명      5722 non-null   object
 2   입항년도    5722 non-null   int64 
 3   입항횟수    5722 non-null   int64 
 4   입출      5722 non-null   int64 
 5   총톤수     5722 non-null   object
 6   입항일시    5722 non-null   object
 7   출항일시    5722 non-null   object
 8   MRN 번호  5719 non-null   object
 9   계선장소    5722 non-null   object
 10  계선장소.1  5722 non-null   int64 
 11  차항지     5720 non-null   object
 12  전출항지    5720 non-null   object
 13  예선      5720 non-null   object
 14  도선      5720 non-null   object
dtypes: int64(4), object(11)
memory usage: 715.2+ KB


In [106]:
In.isnull().sum()

호출부호      0
선명        0
입항년도      0
입항횟수      0
입출        0
총톤수       0
입항일시      0
출항일시      0
MRN 번호    3
계선장소      0
계선장소.1    0
차항지       2
전출항지      2
예선        2
도선        2
dtype: int64

# 입항, 출항 화물료 전처리

### 입항

In [114]:
load_in

,입출항일자,호출부호,선명,입항횟수,선사대리점,납부기한.1,화물료,화물보안료
0,20120101,VREZ4,ATLANTIC MUSE,1,(주)정일스톨트헤븐울,"3,319,360","3,319,360",0
1,20120101,/D8BP,삼영호,1,엘에스니꼬 동제련(주),"1,291,730","1,291,730",0
2,20120101,XUGE6,에버부산,1,동원로엑스(주),"279,310","279,310",0
3,20120101,VRCL5,BEECH GALAXY,1,주식회사태영인더스트리,"159,490","159,490",0
4,20120101,ZDHW2,MATTYS,1,고려해운(주),0,0,0
...,...,...,...,...,...,...,...,...
70753,20210818,DSQP9,온산가스,8,(주)신흥사,"221,560","212,010","9,550"
70754,20210819,9HPO9,CESTENI,1,(주)신흥사,"46,160","44,170","1,990"
70755,20210819,9V8968,WAN HAI 105,2,만해항운한국(주),0,0,0
70756,20210819,9V8968,WAN HAI 105,2,고려해운(주),0,0,0


In [123]:
# 필요없는 컬럼 삭제
load_in.drop(['수리일자', '신고일자', '납부기한'], axis=1, inplace=True)
load_out.drop(['수리일자', '신고일자', '납부기한'], axis=1, inplace=True)
load_out

,입출항일자,호출부호,선명,입항횟수,선사대리점,납부기한.1,화물료,화물보안료
0,20120101,DSQC6,새한 캠스타,14,고려아연(주),"315,940","315,940",0
1,20120101,D7EF,우림,39,에쓰오일 주식회사,"763,140","763,140",0
2,20120101,V7SE3,SEMUA PERKASA,3,에쓰오일 주식회사,"808,710","808,710",0
3,20120101,VRIW3,SAKURA EXPRESS,2,에쓰오일 주식회사,"3,520,860","3,520,860",0
4,20120101,ZDKP2,IVER BLESSING,1,(주)코리아쉽핑트레이,0,0,0
...,...,...,...,...,...,...,...,...
101275,20210819,9V8968,WAN HAI 105,2,만해항운한국(주),"52,060","52,060",0
101276,20210819,3EKP7,SPRING EURO,29,(주)정일스톨트헤븐울,"244,640","244,640",0
101277,20210819,DSDD6,제105효동케미,11,(주)정일스톨트헤븐울,"193,360","193,360",0
101278,20210819,3FBF6,ANGEL BLUE,46,협신컴퍼니,"84,910",NaN,NaN


In [124]:
# 컬럼명 오류난 부분 수정
load_in.rename(columns={'납부기한.1': '고지금액'}, inplace=True)
load_out.rename(columns={'납부기한.1': '고지금액'}, inplace=True)
load_out

,입출항일자,호출부호,선명,입항횟수,선사대리점,고지금액,화물료,화물보안료
0,20120101,DSQC6,새한 캠스타,14,고려아연(주),"315,940","315,940",0
1,20120101,D7EF,우림,39,에쓰오일 주식회사,"763,140","763,140",0
2,20120101,V7SE3,SEMUA PERKASA,3,에쓰오일 주식회사,"808,710","808,710",0
3,20120101,VRIW3,SAKURA EXPRESS,2,에쓰오일 주식회사,"3,520,860","3,520,860",0
4,20120101,ZDKP2,IVER BLESSING,1,(주)코리아쉽핑트레이,0,0,0
...,...,...,...,...,...,...,...,...
101275,20210819,9V8968,WAN HAI 105,2,만해항운한국(주),"52,060","52,060",0
101276,20210819,3EKP7,SPRING EURO,29,(주)정일스톨트헤븐울,"244,640","244,640",0
101277,20210819,DSDD6,제105효동케미,11,(주)정일스톨트헤븐울,"193,360","193,360",0
101278,20210819,3FBF6,ANGEL BLUE,46,협신컴퍼니,"84,910",NaN,NaN


In [125]:
# 입출항일자에서 입항년도 컬럼 생성
load_in['입항년도'] = load_in['입출항일자'].astype(str).str[:4].astype(int)
load_out['입항년도'] = load_out['입출항일자'].astype(str).str[:4].astype(int)
load_out

,입출항일자,호출부호,선명,입항횟수,선사대리점,고지금액,화물료,화물보안료,입항년도
0,20120101,DSQC6,새한 캠스타,14,고려아연(주),"315,940","315,940",0,2012
1,20120101,D7EF,우림,39,에쓰오일 주식회사,"763,140","763,140",0,2012
2,20120101,V7SE3,SEMUA PERKASA,3,에쓰오일 주식회사,"808,710","808,710",0,2012
3,20120101,VRIW3,SAKURA EXPRESS,2,에쓰오일 주식회사,"3,520,860","3,520,860",0,2012
4,20120101,ZDKP2,IVER BLESSING,1,(주)코리아쉽핑트레이,0,0,0,2012
...,...,...,...,...,...,...,...,...,...
101275,20210819,9V8968,WAN HAI 105,2,만해항운한국(주),"52,060","52,060",0,2021
101276,20210819,3EKP7,SPRING EURO,29,(주)정일스톨트헤븐울,"244,640","244,640",0,2021
101277,20210819,DSDD6,제105효동케미,11,(주)정일스톨트헤븐울,"193,360","193,360",0,2021
101278,20210819,3FBF6,ANGEL BLUE,46,협신컴퍼니,"84,910",NaN,NaN,2021


In [137]:
# 날짜데이터를 timestamp(datetime)형으로 변환
load_in['입출항일자'] = load_in['입출항일자'].astype(str)
load_out['입출항일자'] = load_out['입출항일자'].astype(str)
load_in['입출항일자'] = pd.to_datetime(load_in['입출항일자'])
load_out['입출항일자'] = pd.to_datetime(load_out['입출항일자'])
load_out

,입출항일자,호출부호,선명,입항횟수,선사대리점,고지금액,화물료,화물보안료,입항년도
0,2012-01-01,DSQC6,새한 캠스타,14,고려아연(주),"315,940","315,940",0,2012
1,2012-01-01,D7EF,우림,39,에쓰오일 주식회사,"763,140","763,140",0,2012
2,2012-01-01,V7SE3,SEMUA PERKASA,3,에쓰오일 주식회사,"808,710","808,710",0,2012
3,2012-01-01,VRIW3,SAKURA EXPRESS,2,에쓰오일 주식회사,"3,520,860","3,520,860",0,2012
4,2012-01-01,ZDKP2,IVER BLESSING,1,(주)코리아쉽핑트레이,0,0,0,2012
...,...,...,...,...,...,...,...,...,...
101275,2021-08-19,9V8968,WAN HAI 105,2,만해항운한국(주),"52,060","52,060",0,2021
101276,2021-08-19,3EKP7,SPRING EURO,29,(주)정일스톨트헤븐울,"244,640","244,640",0,2021
101277,2021-08-19,DSDD6,제105효동케미,11,(주)정일스톨트헤븐울,"193,360","193,360",0,2021
101278,2021-08-19,3FBF6,ANGEL BLUE,46,협신컴퍼니,"84,910",NaN,NaN,2021


In [138]:
load_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70758 entries, 0 to 70757
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   입출항일자   70758 non-null  datetime64[ns]
 1   호출부호    70758 non-null  object        
 2   선명      70758 non-null  object        
 3   입항횟수    70758 non-null  int64         
 4   선사대리점   70758 non-null  object        
 5   고지금액    70752 non-null  object        
 6   화물료     70748 non-null  object        
 7   화물보안료   70748 non-null  object        
 8   입항년도    70758 non-null  int32         
 9   d       70758 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(6)
memory usage: 5.7+ MB


In [110]:
load_in.isnull().sum()

입출항일자      0
호출부호       0
선명         0
입항횟수       0
선사대리점      0
수리일자      10
신고일자       0
납부기한       0
납부기한.1     6
화물료       10
화물보안료     10
dtype: int64

### 출항

In [23]:
# 입출항일자에서 입항년도 컬럼 생성
df3['입항년도'] = df3['입출항일자'].astype(str).str[:4].astype(int)
df3

,입출항일자,호출부호,선명,입항횟수,선사대리점,수리일자,신고일자,납부기한,납부기한.1,화물료,화물보안료,입항년도
0,20120101,DSQC6,새한 캠스타,14,고려아연(주),20120112.0,20120112,20120116,"315,940","315,940",0,2012
1,20120101,D7EF,우림,39,에쓰오일 주식회사,20120105.0,20120105,20120115,"763,140","763,140",0,2012
2,20120101,V7SE3,SEMUA PERKASA,3,에쓰오일 주식회사,20120105.0,20120105,20120115,"808,710","808,710",0,2012
3,20120101,VRIW3,SAKURA EXPRESS,2,에쓰오일 주식회사,20120105.0,20120105,20120116,"3,520,860","3,520,860",0,2012
4,20120101,ZDKP2,IVER BLESSING,1,(주)코리아쉽핑트레이,20120102.0,20120102,20120116,0,0,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
101275,20210819,9V8968,WAN HAI 105,2,만해항운한국(주),20210820.0,20210820,20210918,"52,060","52,060",0,2021
101276,20210819,3EKP7,SPRING EURO,29,(주)정일스톨트헤븐울,20210820.0,20210820,20210903,"244,640","244,640",0,2021
101277,20210819,DSDD6,제105효동케미,11,(주)정일스톨트헤븐울,20210820.0,20210820,20210903,"193,360","193,360",0,2021
101278,20210819,3FBF6,ANGEL BLUE,46,협신컴퍼니,NaN,20210820,20210903,"84,910",NaN,NaN,2021


# 자동차와 입항, 출항 데이터 각각 merge

In [43]:
# df2 In

test_A = pd.merge(In, df2, on=["호출부호", "선명", "입항횟수", "입항년도"])

In [58]:
test_A

,호출부호,선명,입항년도,입항횟수,입출,총톤수,국제톤수,징수톤수,입항일시,출항일시,...,예선,도선,입출항일자,선사대리점,수리일자,신고일자,납부기한,납부기한.1,화물료,화물보안료
0,V7FT4,NORTHERN HIGHWAY,2012,1,1,"39,422","39,422","39,422",2012-01-02 07:35,2012-01-03 21:35,...,Y,Y,20120102,현대글로비스주식회사,20120105.0,20120105,20120118,"2,940","2,940",0
1,DSMZ8,아세안 메저스티호,2012,1,1,"71,383","71,383","71,383",2012-01-03 10:05,2012-01-03 18:00,...,Y,Y,20120103,유코카캐리어스(주),20120104.0,20120104,20120118,"5,150","5,150",0
2,C6UI2,모닝쿠리어,2012,1,1,"57,692","57,692","57,692",2012-01-06 07:30,2012-01-06 20:25,...,Y,Y,20120106,유코카캐리어스(주),20120109.0,20120109,20120205,"457,050","457,050",0
3,3ETI,MORNING LINDA,2012,1,1,"68,701","68,701","68,701",2012-01-07 07:20,2012-01-07 20:50,...,Y,Y,20120107,현대글로비스주식회사,20120111.0,20120111,20120122,"13,060","13,060",0
4,D8HO,GLOVIS PACIFIC,2012,1,1,"47,367","47,367","47,367",2012-01-11 07:50,2012-01-12 10:45,...,Y,Y,20120111,현대글로비스주식회사,20120113.0,20120113,20120127,"21,520","21,520",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,D8MO,MORNING LAURA,2021,3,1,"70,853","70,853","70,853",2021-07-28 21:55,2021-07-30 10:55,...,Y,Y,20210728,유코카캐리어스(주),20210803.0,20210803,20210814,"420,140","411,660","8,480"
829,V7HR6,VIKING DESTINY,2021,2,1,"62,105","62,105","62,105",2021-07-31 12:10,2021-08-02 20:00,...,Y,Y,20210731,현대글로비스주식회사,20210804.0,20210804,20210817,"156,550","156,550",0
830,SHJC,UNDINE,2021,1,1,"67,378","67,378","67,378",2021-08-11 10:20,2021-08-12 22:00,...,Y,Y,20210811,유코카캐리어스(주),20210819.0,20210818,20210827,"1,202,640","1,178,350","24,290"
831,3EYN5,월러스 에이스,2021,3,1,"63,116","63,116","63,116",2021-08-12 06:00,2021-08-12 18:00,...,Y,Y,20210812,현대글로비스주식회사,20210819.0,20210818,20210829,"3,334,110","3,266,760","67,350"


In [44]:
# df3 Out

test_B = pd.merge(Out, df3, on=["호출부호", "선명", "입항횟수", "입항년도"])

# 적재톤 데이터 전처리

In [41]:
Load_Data =pd.read_excel('data/입출항적재량.xlsx', usecols=[2, 3, 4, 5, 6, 8, 9, 10, 11, 14])
Load_Data

,항차,일시,화물명,적재톤수,양하톤수,출항일시,화물명.1,출항_적재톤수,출항_적하톤수,선명
0,2017-001,2017-07-02 08:00,NaN,0,0,2017-07-02 17:10,차량,800,800,TULANE
1,2019-001,2019-02-02 09:10,차량,7400,0,2019-02-02 16:35,차량,7974,574,TULANE
2,2020-001,2020-01-13 07:20,차량,1482,0,2020-01-13 12:35,차량,2040,558,TULANE
3,2013-001,2013-01-17 12:10,NaN,0,0,2013-01-17 21:30,차량,4466,2466,GLOVIS CARAVEL
4,2013-002,2013-03-28 07:20,NaN,0,0,2013-03-30 16:55,차량,4159,4159,GLOVIS CARAVEL
...,...,...,...,...,...,...,...,...,...,...
5365,2021-001,2021-06-17 11:30,차량,4676,384,2021-06-18 21:00,차량,9286,4994,GRANDE CALIFORNIA
5366,2021-001,2021-06-29 16:50,차량,1199,607,2021-07-01 16:00,차량,6835,5636,탈리아
5367,2021-001,2021-07-03 17:42,NaN,0,0,2021-07-03 21:05,차량,402,402,PASSAMA
5368,2021-002,2021-08-15 08:50,NaN,0,0,2021-08-15 12:30,차량,3000,3000,PASSAMA


In [47]:
# 칼럼 이름 변경
Load_Data.rename(columns={'일시': '입항일시', '항차':'입항횟수'}, inplace=True)
Load_Data

,입항횟수,입항일시,화물명,적재톤수,양하톤수,출항일시,화물명.1,출항_적재톤수,출항_적하톤수,선명
0,2017-001,2017-07-02 08:00,NaN,0,0,2017-07-02 17:10,차량,800,800,TULANE
1,2019-001,2019-02-02 09:10,차량,7400,0,2019-02-02 16:35,차량,7974,574,TULANE
2,2020-001,2020-01-13 07:20,차량,1482,0,2020-01-13 12:35,차량,2040,558,TULANE
3,2013-001,2013-01-17 12:10,NaN,0,0,2013-01-17 21:30,차량,4466,2466,GLOVIS CARAVEL
4,2013-002,2013-03-28 07:20,NaN,0,0,2013-03-30 16:55,차량,4159,4159,GLOVIS CARAVEL
...,...,...,...,...,...,...,...,...,...,...
5365,2021-001,2021-06-17 11:30,차량,4676,384,2021-06-18 21:00,차량,9286,4994,GRANDE CALIFORNIA
5366,2021-001,2021-06-29 16:50,차량,1199,607,2021-07-01 16:00,차량,6835,5636,탈리아
5367,2021-001,2021-07-03 17:42,NaN,0,0,2021-07-03 21:05,차량,402,402,PASSAMA
5368,2021-002,2021-08-15 08:50,NaN,0,0,2021-08-15 12:30,차량,3000,3000,PASSAMA


In [55]:
Load_Data["입항횟수"] = Load_Data["입항횟수"].str[5:].astype(int)

# 적재톤 데이터 merge

In [63]:
InData = pd.merge(test_A, Load_Data, on=["선명", "입항횟수", "입항일시"], how='outer')

In [64]:
OutData = pd.merge(test_B, Load_Data, on=["선명", "입항횟수", "입항일시"], how='outer')

In [65]:
InData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5424 entries, 0 to 5423
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   호출부호     833 non-null    object 
 1   선명       5424 non-null   object 
 2   입항년도     833 non-null    float64
 3   입항횟수     5424 non-null   int64  
 4   입출       833 non-null    float64
 5   총톤수      833 non-null    object 
 6   국제톤수     833 non-null    object 
 7   징수톤수     833 non-null    object 
 8   입항일시     5423 non-null   object 
 9   출항일시_x   833 non-null    object 
 10  CIO수속일자  833 non-null    object 
 11  수리일시     830 non-null    object 
 12  MRN 번호   833 non-null    object 
 13  계선장소     833 non-null    object 
 14  계선장소.1   833 non-null    float64
 15  차항지      833 non-null    object 
 16  전출항지     833 non-null    object 
 17  선박용도     833 non-null    object 
 18  예선       833 non-null    object 
 19  도선       833 non-null    object 
 20  입출항일자    833 non-null    float64
 21  선사대리점    833 n

In [ ]:
df.to_excel(excel_writer='data/PORT_MIS_MERGE.xlsx') #엑셀로 저장